<h1>Sales Analysis</h1>

In [6]:
import pandas as pd
from IPython.display import display
import os

<h2>Merging 12 months of sales data into a single file </h2>

In [26]:
#files = []
#for file in os.listdir('./Sales_Data'):
#    files.append(pd.read_csv(f'./Sales_Data/{file}'))

files = [pd.read_csv(f'./Sales_Data/{file}') for file in os.listdir('./Sales_Data')]

allMonthsDF = pd.concat(files)
allMonthsDF.to_csv('allMonthsDF.csv')

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
11681,259353,AAA Batteries (4-pack),3,2.99,09/17/19 20:56,"840 Highland St, Los Angeles, CA 90001"
11682,259354,iPhone,1,700,09/01/19 16:00,"216 Dogwood St, San Francisco, CA 94016"
11683,259355,iPhone,1,700,09/23/19 07:39,"220 12th St, San Francisco, CA 94016"
11684,259356,34in Ultrawide Monitor,1,379.99,09/19/19 17:30,"511 Forest St, San Francisco, CA 94016"


<h3>Read the updated csv</h3>

In [31]:
all_data = pd.read_csv('allMonthsDF.csv')

<h2> Question 1: What was the best month for sales? How much was earned that month?</h2>

<h3> The best month for sales

In [61]:
selected_sales_data = all_data[['Order Date', 'Quantity Ordered', 'Price Each']]
selected_sales_data['Order Date']


TypeError: 'DataFrame' object is not callable